<a href="https://colab.research.google.com/github/mbCap/fallstudie_BESS/blob/main/Fallstudie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Master Planning

In [74]:
! git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

fatal: destination path 'WS24_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [75]:
! pip install -q pyscipopt

In [76]:
import pandas as pd
from pyscipopt import Model, quicksum

## Daten laden

In [77]:
folder = "WS24_Supply_Chain_Optimierung/Daten/Fallstudie/"

In [78]:
path = pd.read_excel(f"{folder}/Preisprognosen.xlsx",)

## Indexmengen

In [79]:
T = path["Stunde"].tolist()  # Stunden des Folgetages aus den Daten

In [80]:
D = ["buy", "sell"]  # Entscheidungen: Kauf und Verkauf

Wahrscheinlichkeiten

In [81]:
# Wahrscheinlichkeiten aus der Excel-Datei extrahieren
P_prob = path['Wahrscheinlichkeit'].tolist()

# Überprüfen, ob die Anzahl der Wahrscheinlichkeiten und Stunden übereinstimmt
assert len(P_prob) == len(T), "Die Anzahl der Stunden und Wahrscheinlichkeiten muss übereinstimmen"

## Optimierungsmodell

In [82]:
scip = Model("BESS_Optimization")

## Parameter

In [83]:
C_nom = 40  # Nominelle Speicherkapazität (MWh)

In [84]:
DoD = 0.8  # Depth of Discharge

In [85]:
SOC_min = (1 - DoD) * C_nom  # Minimaler Ladezustand (MWh)

In [86]:
SOC_max = C_nom  # Maximaler Ladezustand (MWh)

In [87]:
C_rate = 0.5 * C_nom  # Maximale Lade-/Entladeleistung (MWh/h)

In [88]:
eta_RT = 0.95  # Round-Trip-Effizienz

In [89]:
eta_inv = 0.985  # Wechselrichter-Wirkungsgrad

In [90]:
Z_max = 2  # Maximale Anzahl von Vollzyklen

In [91]:
C_cycle = 1500  # Kosten pro Speicherzyklus (€)

In [92]:
P_min = -500  # Minimaler Gebotspreis (€/MWh)

In [93]:
P_max = 4000  # Maximaler Gebotspreis (€/MWh)

Entscheidungsvariablen

In [94]:
Q = {(t, d): scip.addVar(lb=0, name=f"Q_{t}_{d}") for t in T for d in D}  # Gebotsmengen (MWh)

In [95]:
P = {(t, d): scip.addVar(lb=P_min, ub=P_max, name=f"P_{t}_{d}") for t in T for d in D}  # Gebotspreise (€/MWh)

In [96]:
SOC = {t: scip.addVar(lb=SOC_min, ub=SOC_max, name=f"SOC_{t}") for t in T}  # Ladezustand (MWh)

In [97]:
Z = {t: scip.addVar(vtype="B", name=f"Z_{t}") for t in T}  # Speicherzyklen-Binärvariable

In [98]:
# Neue Variablen X für Produkte von P[t,d] und Q[t,d]
X = {(t, d): scip.addVar(lb=0, name=f"X_{t}_{d}") for t in T for d in D}

## Zielfunktion

$\max \sum_{t \in T} \sum_{d \in D} \left( P_{t,d} \cdot Q_{t,d} \cdot \mathbb{P}(P_{\text{market}} = P_{t,d}) \right) - \sum_{t \in T} C_{\text{cycle}} \cdot Z_t$

In [99]:
# Zielfunktion mit den neuen Variablen X
scip.setObjective(
    sum(P_prob[t] * X[t, d] for t in T for d in D) - sum(C_cycle * Z[t] for t in T),
    "maximize"
)

## Nebenbedingungen

Ladezustandsdynamik

In [100]:
for t in T:
    if t == T[0]:
        scip.addCons(SOC[t] == SOC_min)  # Initialer Ladezustand
    else:
        scip.addCons(
            SOC[t] == SOC[T[T.index(t)-1]] + eta_inv * Q[t, "buy"] - (1 / (eta_RT * eta_inv)) * Q[t, "sell"]
        )

Leistungsbeschränkung

In [101]:
for t in T:
    scip.addCons(Q[t, "buy"] <= C_rate)
    scip.addCons(Q[t, "sell"] <= C_rate)

Kapazitätsbeschränkung

In [102]:
for t in T:
    scip.addCons(SOC[t] + Q[t, "buy"] <= SOC_max)
    scip.addCons(SOC[t] - Q[t, "sell"] >= SOC_min)

Max Anzahl Zyklen

In [103]:
scip.addCons(sum(Z[t] for t in T) <= Z_max)

c3001

Def der Zyklen

In [104]:
for t in T:
    scip.addCons(Z[t] >= (Q[t, "buy"] + Q[t, "sell"]) / C_nom)  # Zyklenbeginn

## Berechnung Lösung

In [105]:
scip.optimize()

In [106]:
print("\nErgebnisse:")
if scip.getStatus() == "optimal":
    print(f"Maximierter Gewinn: {scip.getObjVal():.2f} €")
    print("\nGebote und Ladezustand pro Stunde:")
    for t in T:
        print(f"Stunde {t}: Buy {scip.getVal(Q[t, 'buy']):.2f} MWh, "
              f"Sell {scip.getVal(Q[t, 'sell']):.2f} MWh, "
              f"SOC: {scip.getVal(SOC[t]):.2f} MWh")
else:
    print("Keine optimale Lösung gefunden.")


Ergebnisse:
Keine optimale Lösung gefunden.
